In [101]:
from collections import Counter
from datetime import datetime, timedelta as td
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import sys

dtf = datetime.strftime
dtp = datetime.strptime

In [102]:
class ExpLogHandler:
    """Class to find, combine, and pre-process all required data files.

    Args:
        log_names (list): List of names of required log files.

    Attributes:
        log_names (list): Where log_names is stored.
    """
    def __init__(self, log_names):
        """Initialiser for the ExpLogHandler class.

        Args:
            log_names (list): List of names of required log files.
        """
        self.log_names = log_names
        
    def getData(self):
        """Acquire data files, combine, remove duplicate column headers, join indices

        Args:
            None
        
        Returns:
            data: Complete DataFrame of all experiment log lines.
        """

        self.data = pd.read_csv(self.log_names[0], on_bad_lines="skip", dtype={'SDC_val': str})
        for i in range(1, len(self.log_names)):
            data_next   = pd.read_csv(self.log_names[i], on_bad_lines="skip", dtype={'SDC_val': str})
            data_frames = [self.data, data_next]
            self.data   = pd.concat(data_frames, ignore_index=True)
        self.data.drop(self.data.loc[(self.data["Timestamp"] == "Timestamp")].index.tolist())
        self.dataLength = self.data.shape[0]

        wrchckbrdLocs  = self.data.loc[(self.data["Board"] == "M") & (self.data["Mikroe_socket"] == "A") &
                                       (self.data["Status"] == "WRCHCKBRD")].index.tolist()
        wrchckbrdIndex = [(i, "WRCHCKBRD") for i in wrchckbrdLocs]
        initLocs       = self.data.loc[(self.data["Board"] == "M") & (self.data["Mikroe_socket"] == "A") &
                         (self.data["Status"] == "INIT")].index.tolist()
        initIndex      = [(i, "INIT") for i in initLocs]
        runIndex       = wrchckbrdIndex + initIndex
        runIndex       = sorted(runIndex)
        runIndex.append((self.dataLength, "END"))

        runId  = 1
        runCol = [''] * self.dataLength
        for h in range(len(runIndex) - 1):
            if runIndex[h][1] == "WRCHCKBRD":
                start              = runIndex[h][0]
                end                = runIndex[h+1][0]
                runCol[start:end]  = [runId]*(end-start)
                runId              += 1

        self.data.insert(loc=0, column="Run_ID", value=runCol)
        return self.data

In [103]:
class BeamLogHandler:
    """Class to retrieve all neutron logs for experiment dates.

    Attributes:
        beamLogFnameFormat (str): Datetime format for beamlog file names.
        firstTime (datetime): Datetime object of first experiment timestamp.
        lastTime (datetime): Datetime object of last experiment timestamp.
        targetDir_fullpathList (list): List of each parent directory of experiment results directory.
    
    """
    def __init__(self, firstTime, lastTime, targetDir_fullpathList):     
        self.beamLogFnameFormat          = "%Y-%m-%d"
        self.firstTime                   = firstTime
        self.lastTime                    = lastTime
        self.targetDir_fullpathList      = targetDir_fullpathList
        self.targetDir_fullpathList[-4:] = ["neutrons"]

    def getLogs(self):
        firstBeamTime = dtf(self.firstTime, self.beamLogFnameFormat)
        beamlogDayDif = (self.lastTime - self.firstTime).days
        beamPath      = "../../../neutrons/countlog-"
        self.beamlog  = pd.read_csv(f"{beamPath}{firstBeamTime}.txt",
                        delim_whitespace=True, header=None, skiprows=1)
        nextTime      = self.firstTime
        for i in range(0, beamlogDayDif+1):
            nextTime      = nextTime + td(days=1)
            nextBeamTime  = dtf(nextTime, self.beamLogFnameFormat)
            try:
                nextBeamlog   = pd.read_csv(f"{beamPath}{nextBeamTime}.txt",delim_whitespace=True,header=None,skiprows=1)
            except FileNotFoundError:
                pass
            beamlogFrames = [self.beamlog, nextBeamlog]
            self.beamlog  = pd.concat(beamlogFrames, ignore_index=True)
            
        self.beamlog.columns = ["Date","HMS_time","Millisecs","Count1","Count2",
                                "Count3","Count4","protonCharge","Beam_current"]
        return self.beamlog

In [104]:
class Run:
    """Class for one repetition of an experiment.

    Args:
        num (int): Run number for the selected experiment logs.
        runData (DataFrame): Truncated DataFrame of log lines for the run.
    
    Attributes:
        delay (int): Length in milliseconds of beam 'bathing time'.
        df (DataFrame): Saving runData.
        errors (int): Number of nvRAM errors.
        fullErr (int): 1 if the entire nvRAM array was errored.
        hot (int): 1 if the beam was on, 0 if not.
        num (int): Saving num.
        rowErr (int): 1 if there was at least one row error.
        see (int): Number of true SEEs detected.
        valid (int): 1 if the run had all expected log lines, 0 if not.
        _ErrorsNVRAM (list): List of tuples (error_byte, error_location) for each chip.
    """
    def __init__(self, num, runData):
        """Initialiser for the Run class. 

        Parameters:
            num (int): Run number for the selected experiment logs.
            runData (DataFrame): Truncated DataFrame of log lines for the run.
        """
        self.delay     = -1
        self.df        = runData
        self.errors    = 0
        self.fullErr   = -1
        self.hot       = -1
        self.num       = num
        self.rowErr    = -1
        self.see       = -1
        self.valid     = -1
        self.aErrNVRAM = []
        self.bErrNVRAM = []
        self.cErrNVRAM = []
        self.dErrNVRAM = []
        self.aErrorBounds = []
        self.bErrorBounds = []
        self.cErrorBounds = []
        self.dErrorBounds = []
        self.aErrNVRAMVals = []
        self.bErrNVRAMVals = []
        self.cErrNVRAMVals = []
        self.dErrNVRAMVals = []
        self.aErrNVRAMLocs = []
        self.bErrNVRAMLocs = []
        self.cErrNVRAMLocs = []
        self.dErrNVRAMLocs = []

In [105]:
class Analyser:
    '''
    Class to handle processing data.

    Args:
        targetDir_fullpath (string): File path of directory to analyse.

    Attributes:
        targetDir_fullpath (string): Storing targetDir_fullpath arg.
        expLogTstampFormat (string): Datetime format for experiment log timestamps
        beamLogTstampFormat (string): Datetime format for beam log timestamps                
    '''
    def __init__(self, targDir_full):
        '''
        Retrieve + store target directory, initialise datetime formats.

        Args:
            targetDir_fullpath (string): File path of directory to analyse.

        Attributes:
            targetDir_fullpath (string): Storing targetDir_fullpath arg.
            expLogTstampFormat (string): Datetime format for experiment log timestamps
            beamLogTstampFormat (string): Datetime format for beam log timestamps
        '''
        #self.targetDir_fullpath  = targetDir_fullpath
        self.targDir_full = targDir_full
        self.expLogTstampFormat  = "%Y-%m-%d_%H-%M-%S-%f"
        self.beamLogTstampFormat = "%d/%m/%Y %H:%M:%S"
 
    def setup(self):
        #targetDir_fullpathList = self.targDir_full.split("/")
        targDir_fullList = self.targDir_full.split("/")
        targDir_trunc    = targDir_fullList[-4:-1]
        self.chip        = targDir_trunc[0]
        self.variant     = targDir_trunc[1]
        self.size        = targDir_trunc[2]

        # Get data from logs
        self.log_names     = [self.targDir_full+i for i in os.listdir(self.targDir_full) if ".csv" in i]
        self.log_names.sort(key = lambda x: dtp(x, f"{self.targDir_full}nvRAM_data_%d-%m-%Y-%H%M.csv"))
        self.ExpLogHandler = ExpLogHandler(self.log_names)
        self.data          = self.ExpLogHandler.getData()

        # Get beam status from logs
        firstTimestamp      = self.data[0:1]['Timestamp'][0]
        lastTimestamp       = self.data[self.data.shape[0]-2:self.data.shape[0]-1]['Timestamp'][self.data.shape[0]-2]
        firstTime           = dtp(firstTimestamp, self.expLogTstampFormat)
        lastTime            = dtp(lastTimestamp, self.expLogTstampFormat)
        self.BeamLogHandler = BeamLogHandler(firstTime, lastTime, targDir_fullList)
        self.beamlog        = self.BeamLogHandler.getLogs()

        # Create datetime objects from beamlog timestamps
        beamlogTstamps    = (self.beamlog['Date']+self.beamlog['HMS_time']+self.beamlog['Millisecs'].apply(str)).tolist()
        self.beamlogTimes = []
        for i in beamlogTstamps:
            if len(i) > 23:
                i = i[0:26]
            self.beamlogTimes.append(i)
        self.beamlogTimes = [dtp(i, '%d/%m/%Y%H:%M:%S0.%f') for i in self.beamlogTimes]

    def beamOn(self, firstTstamp, lastTstamp):
        bOfirstTime = firstTstamp
        bOlastTime = lastTstamp
        beamTimeNearFirstTime = min([i for i in self.beamlogTimes if i <= bOfirstTime], key=lambda x: abs(x - bOfirstTime))
        beamTimeNearLastTime  = min([i for i in self.beamlogTimes if i >= bOlastTime], key=lambda x: abs(x - bOlastTime))
        firstRow              = self.beamlog.loc[(self.beamlog['Date'] == dtf(beamTimeNearFirstTime, '%d/%m/%Y')) &
                                (self.beamlog['HMS_time'] == dtf(beamTimeNearFirstTime, '%H:%M:%S'))]
        lastRow               = self.beamlog.loc[(self.beamlog['Date'] == dtf(beamTimeNearLastTime, '%d/%m/%Y')) & 
                                (self.beamlog['HMS_time'] == dtf(beamTimeNearLastTime, '%H:%M:%S'))]
        count4Dif             = lastRow.iloc[0]['Count4'] - firstRow.iloc[0]['Count4']
        numRows               = lastRow.index.astype(int)[0] - firstRow.index.astype(int)[0]
        cps                   = count4Dif / numRows
        if cps > 1:
            return 1
        return 0

    def createRuns(self):
        runNames  = []
        self.runs = {}
        lastRunID =  self.data[self.data['Run_ID'].last_valid_index():self.data['Run_ID'].
                     last_valid_index()+1]['Run_ID'][self.data['Run_ID'].last_valid_index()]

        # Supply Run initialiser with truncated data 
        for i in range(1, lastRunID + 1):
            runNames.append(f"run_{i}")
            runData      = self.data.loc[(self.data["Run_ID"] == i)]
            runData.reset_index(inplace=True, drop=True)
            self.runs[i] = Run(i, runData)

        for run in self.runs.values():
            if self.variant == "1":
                    try:
                        # Extract beam delay from timestamps
                        before_delay_i  = run.df.loc[(run.df['Mikroe_socket']=='D') & (run.df['Status']=='STORE_OK')].index[0]
                        before_delay_t  = run.df.loc[before_delay_i]['Timestamp']
                        after_delay_i   = run.df.loc[(run.df['Mikroe_socket']=='A') & (run.df['Status']=='VERIF')].index[0] + 1
                        after_delay_t   = run.df.loc[after_delay_i]['Timestamp']
                        before_delay_dt = dtp(before_delay_t, self.expLogTstampFormat)
                        after_delay_dt  = dtp(after_delay_t, self.expLogTstampFormat)   
                        dif             = after_delay_dt - before_delay_dt
                        
                        if td(seconds = 0.1) < dif < td(seconds = 0.5):
                            run.delay = 0.1
                        if td(seconds = 0.5) < dif < td(seconds = 5):
                            run.delay = 1
                        if td(seconds = 5)   < dif < td(seconds = 50):
                            run.delay = 10
                        if td(seconds = 50)  < dif < td(seconds = 500):
                            run.delay = 100
                        if td(seconds = 500) < dif < td(seconds = 5000):
                            run.delay = 1000

                        # Was the beam on?
                        try:
                            if self.beamOn(before_delay_dt, after_delay_dt):
                                run.hot = 1
                            else:
                                run.hot = 0
                        except ValueError:
                            print(run.name)

                        # Create list of errors in nvRAM per chip
                        aErrorBounds = list(zip(run.df.loc[(run.df["Mikroe_socket"] == "A") & 
                                            (run.df["Status"] == "VERIF")].index.tolist(),
                                                run.df.loc[(run.df["Mikroe_socket"] == "A") & 
                                            (run.df["Status"] == "VERIF_OK")].index.tolist()))

                        bErrorBounds = list(zip(run.df.loc[(run.df["Mikroe_socket"] == "B") & 
                                            (run.df["Status"] == "VERIF")].index.tolist(),
                                                run.df.loc[(run.df["Mikroe_socket"] == "B") &
                                            (run.df["Status"] == "VERIF_OK")].index.tolist()))

                        cErrorBounds = list(zip(run.df.loc[(run.df["Mikroe_socket"] == "C") & 
                                            (run.df["Status"] == "VERIF")].index.tolist(),
                                                run.df.loc[(run.df["Mikroe_socket"] == "C") & 
                                            (run.df["Status"] == "VERIF_OK")].index.tolist()))

                        dErrorBounds = list(zip(run.df.loc[(run.df["Mikroe_socket"] == "D") & 
                                            (run.df["Status"] == "VERIF")].index.tolist(),
                                                run.df.loc[(run.df["Mikroe_socket"] == "D") & 
                                            (run.df["Status"] == "VERIF_OK")].index.tolist()))

                        aErrorsBefRec = list(zip(run.df[aErrorBounds[0][0]+1:aErrorBounds[0][1]]["SDC_val"].values.tolist(),
                                                run.df[aErrorBounds[0][0]+1:aErrorBounds[0][1]]["SDC_loc"].values.tolist()))
                        aErrorsAftRec = list(zip(run.df[aErrorBounds[1][0]+1:aErrorBounds[1][1]]["SDC_val"].values.tolist(),
                                                run.df[aErrorBounds[1][0]+1:aErrorBounds[1][1]]["SDC_loc"].values.tolist()))

                        bErrorsBefRec = list(zip(run.df[bErrorBounds[0][0]+1:bErrorBounds[0][1]]["SDC_val"].values.tolist(),
                                                run.df[bErrorBounds[0][0]+1:bErrorBounds[0][1]]["SDC_loc"].values.tolist()))
                        bErrorsAftRec = list(zip(run.df[bErrorBounds[1][0]+1:bErrorBounds[1][1]]["SDC_val"].values.tolist(),
                                                run.df[bErrorBounds[1][0]+1:bErrorBounds[1][1]]["SDC_loc"].values.tolist()))

                        cErrorsBefRec = list(zip(run.df[cErrorBounds[0][0]+1:cErrorBounds[0][1]]["SDC_val"].values.tolist(),
                                                run.df[cErrorBounds[0][0]+1:cErrorBounds[0][1]]["SDC_loc"].values.tolist()))
                        cErrorsAftRec = list(zip(run.df[cErrorBounds[1][0]+1:cErrorBounds[1][1]]["SDC_val"].values.tolist(),
                                                run.df[cErrorBounds[1][0]+1:cErrorBounds[1][1]]["SDC_loc"].values.tolist()))

                        dErrorsBefRec = list(zip(run.df[dErrorBounds[0][0]+1:dErrorBounds[0][1]]["SDC_val"].values.tolist(),
                                                run.df[dErrorBounds[0][0]+1:dErrorBounds[0][1]]["SDC_loc"].values.tolist()))
                        dErrorsAftRec = list(zip(run.df[dErrorBounds[1][0]+1:dErrorBounds[1][1]]["SDC_val"].values.tolist(),
                                                run.df[dErrorBounds[1][0]+1:dErrorBounds[1][1]]["SDC_loc"].values.tolist())) 
                        
                        # run.aErrNVRAMVals = [list(i) for i in zip(*[i for i in aErrorsAftRec if i not in aErrorsBefRec if np.nan not in i])][0]
                        # run.bErrNVRAMVals = [list(i) for i in zip(*[i for i in bErrorsAftRec if i not in bErrorsBefRec if np.nan not in i])][0]
                        # run.cErrNVRAMVals = [list(i) for i in zip(*[i for i in cErrorsAftRec if i not in cErrorsBefRec if np.nan not in i])][0]
                        # run.dErrNVRAMVals = [list(i) for i in zip(*[i for i in dErrorsAftRec if i not in dErrorsBefRec if np.nan not in i])][0]

                        run.aErrNVRAM = [i for i in bErrorsAftRec if i not in bErrorsBefRec if np.nan not in i]
                        run.bErrNVRAM = [i for i in bErrorsAftRec if i not in bErrorsBefRec if np.nan not in i]
                        run.cErrNVRAM = [i for i in cErrorsAftRec if i not in cErrorsBefRec if np.nan not in i]
                        run.dErrNVRAM = [i for i in dErrorsAftRec if i not in dErrorsBefRec if np.nan not in i]

                        # run.aErrNVRAMLocs = [int(i, 16) for i in [list(i) for i in zip(*[i for i in aErrorsAftRec if i not in aErrorsBefRec if np.nan not in i])][1]]
                        # run.bErrNVRAMLocs = [int(i, 16) for i in [list(i) for i in zip(*[i for i in bErrorsAftRec if i not in bErrorsBefRec if np.nan not in i])][1]]
                        # run.cErrNVRAMLocs = [int(i, 16) for i in [list(i) for i in zip(*[i for i in cErrorsAftRec if i not in cErrorsBefRec if np.nan not in i])][1]]
                        # run.dErrNVRAMLocs = [int(i, 16) for i in [list(i) for i in zip(*[i for i in dErrorsAftRec if i not in dErrorsBefRec if np.nan not in i])][1]]

                        # run.aErrorsNVRAMDict = {k:v for k, v in sorted(Counter(run.aErrorsNVRAM).items(), reverse=True, key=lambda i: i[1])}
                        # run.bErrorsNVRAMDict = {k:v for k, v in sorted(Counter(run.bErrorsNVRAM).items(), reverse=True, key=lambda i: i[1])}
                        # run.cErrorsNVRAMDict = {k:v for k, v in sorted(Counter(run.cErrorsNVRAM).items(), reverse=True, key=lambda i: i[1])}
                        # run.dErrorsNVRAMDict = {k:v for k, v in sorted(Counter(run.dErrorsNVRAM).items(), reverse=True, key=lambda i: i[1])}
                        
                        run.errors += len(run.aErrNVRAM)
                        run.errors += len(run.bErrNVRAM)
                        run.errors += len(run.cErrNVRAM)
                        run.errors += len(run.dErrNVRAM)

                    except IndexError:
                        run.valid = 0
                    else:
                        run.valid = 1
                        
            if self.variant == "2":
                try:
                    # Extract beam delay from timestamps
                    before_delay_i  = run.df.loc[(run.df['Mikroe_socket']=='D') & (run.df['Status']=='STORE_OK')].index[0]
                    before_delay_t  = run.df.loc[before_delay_i]['Timestamp']
                    after_delay_i   = run.df.loc[(run.df['Mikroe_socket']=='A') & (run.df['Status']=='VERIF')].index[0] + 1
                    after_delay_t   = run.df.loc[after_delay_i]['Timestamp']
                    before_delay_dt = dtp(before_delay_t, self.expLogTstampFormat)
                    after_delay_dt  = dtp(after_delay_t, self.expLogTstampFormat)   
                    dif             = after_delay_dt - before_delay_dt
                    
                    if td(seconds = 0.1) < dif < td(seconds = 0.5):
                        run.delay = 0.1
                    if td(seconds = 0.5) < dif < td(seconds = 5):
                        run.delay = 1
                    if td(seconds = 5)   < dif < td(seconds = 50):
                        run.delay = 10
                    if td(seconds = 50)  < dif < td(seconds = 500):
                        run.delay = 100
                    if td(seconds = 500) < dif < td(seconds = 5000):
                        run.delay = 1000

                    # Was the beam on?
                    try:
                        if self.beamOn(before_delay_dt, after_delay_dt):
                            run.hot = 1
                        else:
                            run.hot = 0
                    except ValueError:
                        print(run.name)
                        
                    # Create list of errors in nvRAM per chip
                    aErrorBounds = list(zip(run.df.loc[(run.df["Mikroe_socket"] == "A") & 
                                        (run.df["Status"] == "VERIF")].index.tolist(),
                                            run.df.loc[(run.df["Mikroe_socket"] == "A") & 
                                        (run.df["Status"] == "VERIF_OK")].index.tolist()))

                    bErrorBounds = list(zip(run.df.loc[(run.df["Mikroe_socket"] == "B") & 
                                        (run.df["Status"] == "VERIF")].index.tolist(),
                                            run.df.loc[(run.df["Mikroe_socket"] == "B") &
                                        (run.df["Status"] == "VERIF_OK")].index.tolist()))

                    cErrorBounds = list(zip(run.df.loc[(run.df["Mikroe_socket"] == "C") & 
                                        (run.df["Status"] == "VERIF")].index.tolist(),
                                            run.df.loc[(run.df["Mikroe_socket"] == "C") & 
                                        (run.df["Status"] == "VERIF_OK")].index.tolist()))

                    dErrorBounds = list(zip(run.df.loc[(run.df["Mikroe_socket"] == "D") & 
                                        (run.df["Status"] == "VERIF")].index.tolist(),
                                            run.df.loc[(run.df["Mikroe_socket"] == "D") & 
                                        (run.df["Status"] == "VERIF_OK")].index.tolist()))

                    aErrBefDelay = list(zip(run.df[aErrorBounds[0][0]+1:aErrorBounds[0][1]]["SDC_val"].values.tolist(),
                                            run.df[aErrorBounds[0][0]+1:aErrorBounds[0][1]]["SDC_loc"].values.tolist()))
                    aErrBefRec = list(zip(run.df[aErrorBounds[1][0]+1:aErrorBounds[1][1]]["SDC_val"].values.tolist(),
                                            run.df[aErrorBounds[1][0]+1:aErrorBounds[1][1]]["SDC_loc"].values.tolist()))
                    aErrAftRec = list(zip(run.df[aErrorBounds[2][0]+1:aErrorBounds[2][1]]["SDC_val"].values.tolist(),
                                            run.df[aErrorBounds[2][0]+1:aErrorBounds[2][1]]["SDC_loc"].values.tolist()))

                    bErrBefDelay = list(zip(run.df[bErrorBounds[0][0]+1:bErrorBounds[0][1]]["SDC_val"].values.tolist(),
                                            run.df[bErrorBounds[0][0]+1:bErrorBounds[0][1]]["SDC_loc"].values.tolist()))
                    bErrBefRec = list(zip(run.df[bErrorBounds[1][0]+1:bErrorBounds[1][1]]["SDC_val"].values.tolist(),
                                            run.df[bErrorBounds[1][0]+1:bErrorBounds[1][1]]["SDC_loc"].values.tolist()))
                    bErrAftRec = list(zip(run.df[bErrorBounds[2][0]+1:bErrorBounds[2][1]]["SDC_val"].values.tolist(),
                                            run.df[bErrorBounds[2][0]+1:bErrorBounds[2][1]]["SDC_loc"].values.tolist()))
                    
                    cErrBefDelay = list(zip(run.df[cErrorBounds[0][0]+1:cErrorBounds[0][1]]["SDC_val"].values.tolist(),
                                            run.df[cErrorBounds[0][0]+1:cErrorBounds[0][1]]["SDC_loc"].values.tolist()))
                    cErrBefRec = list(zip(run.df[cErrorBounds[1][0]+1:cErrorBounds[1][1]]["SDC_val"].values.tolist(),
                                            run.df[cErrorBounds[1][0]+1:cErrorBounds[1][1]]["SDC_loc"].values.tolist()))
                    cErrAftRec = list(zip(run.df[cErrorBounds[2][0]+1:cErrorBounds[2][1]]["SDC_val"].values.tolist(),
                                            run.df[cErrorBounds[2][0]+1:cErrorBounds[2][1]]["SDC_loc"].values.tolist()))
                    
                    dErrBefDelay = list(zip(run.df[dErrorBounds[0][0]+1:dErrorBounds[0][1]]["SDC_val"].values.tolist(),
                                            run.df[dErrorBounds[0][0]+1:dErrorBounds[0][1]]["SDC_loc"].values.tolist()))
                    dErrBefRec = list(zip(run.df[dErrorBounds[1][0]+1:dErrorBounds[1][1]]["SDC_val"].values.tolist(),
                                            run.df[dErrorBounds[1][0]+1:dErrorBounds[1][1]]["SDC_loc"].values.tolist()))
                    dErrAftRec = list(zip(run.df[dErrorBounds[2][0]+1:dErrorBounds[2][1]]["SDC_val"].values.tolist(),
                                            run.df[dErrorBounds[2][0]+1:dErrorBounds[2][1]]["SDC_loc"].values.tolist()))
                    
                    run.aErrNVRAM = [i for i in aErrAftRec if i not in aErrBefDelay if i not in aErrBefRec if np.nan not in i]
                    run.bErrNVRAM = [i for i in bErrAftRec if i not in bErrBefDelay if i not in bErrBefRec if np.nan not in i]
                    run.cErrNVRAM = [i for i in cErrAftRec if i not in cErrBefDelay if i not in cErrBefRec if np.nan not in i]
                    run.dErrNVRAM = [i for i in dErrAftRec if i not in dErrBefDelay if i not in dErrBefRec if np.nan not in i]
                    
                    # run.aErrNVRAMVals = [list(i) for i in zip(*[i for i in aErrAftRec if i not in aErrBefRec if np.nan not in i])][0]
                    # run.bErrNVRAMVals = [list(i) for i in zip(*[i for i in bErrAftRec if i not in bErrBefRec if np.nan not in i])][0]
                    # run.cErrNVRAMVals = [list(i) for i in zip(*[i for i in cErrAftRec if i not in cErrBefRec if np.nan not in i])][0]
                    # run.dErrNVRAMVals = [list(i) for i in zip(*[i for i in dErrAftRec if i not in dErrBefRec if np.nan not in i])][0]

                    # run.aErrNVRAMLocs = [int(i, 16) for i in [list(i) for i in zip(*[i for i in aErrAftRec if i not in aErrBefRec if np.nan not in i])][1]]
                    # run.bErrNVRAMLocs = [int(i, 16) for i in [list(i) for i in zip(*[i for i in bErrAftRec if i not in bErrBefRec if np.nan not in i])][1]]
                    # run.cErrNVRAMLocs = [int(i, 16) for i in [list(i) for i in zip(*[i for i in cErrAftRec if i not in cErrBefRec if np.nan not in i])][1]]
                    # run.dErrNVRAMLocs = [int(i, 16) for i in [list(i) for i in zip(*[i for i in dErrAftRec if i not in dErrBefRec if np.nan not in i])][1]]
                    
                    # # run.aErrorsNVRAMDict = {k:v for k, v in sorted(Counter(run.aErrorsNVRAM).items(), reverse=True, key=lambda i: i[1])}
                    # # run.bErrorsNVRAMDict = {k:v for k, v in sorted(Counter(run.bErrorsNVRAM).items(), reverse=True, key=lambda i: i[1])}
                    # # run.cErrorsNVRAMDict = {k:v for k, v in sorted(Counter(run.cErrorsNVRAM).items(), reverse=True, key=lambda i: i[1])}
                    # # run.dErrorsNVRAMDict = {k:v for k, v in sorted(Counter(run.dErrorsNVRAM).items(), reverse=True, key=lambda i: i[1])}
                
                    run.errors += len(run.aErrNVRAM)
                    run.errors += len(run.bErrNVRAM)
                    run.errors += len(run.cErrNVRAM)
                    run.errors += len(run.dErrNVRAM)

                # Invalidate run if critical experiment steps didn't occur
                except IndexError:
                    run.valid = 0
                else:
                    run.valid = 1

In [106]:
sram3_1_single   = Analyser("C:/Users/Sujit/Documents/STFC/Code/Mikroe/nvRAM_experiment/results/live/SRAM3/1/single/")
sram3_1_single.setup()
sram3_1_single.createRuns()
print("sram3_1_single ready")

sram3_1_half     = Analyser("C:/Users/Sujit/Documents/STFC/Code/Mikroe/nvRAM_experiment/results/live/SRAM3/1/half/")
sram3_1_half.setup()
sram3_1_half.createRuns()
print("sram3_1_half ready")

sram3_2_half     = Analyser("C:/Users/Sujit/Documents/STFC/Code/Mikroe/nvRAM_experiment/results/live/SRAM3/2/half/")
sram3_2_half.setup()
sram3_2_half.createRuns()
print("sram3_2_half ready")

nvsram2_1_single = Analyser("C:/Users/Sujit/Documents/STFC/Code/Mikroe/nvRAM_experiment/results/live/nvSRAM2/1/single/")
nvsram2_1_single.setup()
nvsram2_1_single.createRuns()
print("nvsram2_1_single ready")

nvsram2_1_half   = Analyser("C:/Users/Sujit/Documents/STFC/Code/Mikroe/nvRAM_experiment/results/live/nvSRAM2/1/half/")
nvsram2_1_half.setup()
nvsram2_1_half.createRuns()
print("nvsram2_1_half ready")

nvsram2_2_half   = Analyser("C:/Users/Sujit/Documents/STFC/Code/Mikroe/nvRAM_experiment/results/live/nvSRAM2/2/half/")
nvsram2_2_half.setup()
nvsram2_2_half.createRuns()
print("nvsram2_2_half ready")

sram3_1_single ready
sram3_1_half ready
sram3_2_half ready
nvsram2_1_single ready
nvsram2_1_half ready
nvsram2_2_half ready


In [ ]:
#tdf = nvsram2_2_half.data.loc[(nvsram2_2_half.data['Run_ID'] == 6)]

tdf = nvsram2_2_half.data.loc[(nvsram2_2_half.data['Run_ID'] == 6)]
tdf.reset_index(inplace=True, drop=True)


bErrorBounds = list(zip(tdf.loc[(tdf["Mikroe_socket"] == "B") & 
                                            (tdf["Status"] == "VERIF")].index.tolist(),
                                                tdf.loc[(tdf["Mikroe_socket"] == "B") &
                                            (tdf["Status"] == "VERIF_OK")].index.tolist()))

bErrBefDelay = list(zip(tdf[bErrorBounds[0][0]+1:bErrorBounds[0][1]]["SDC_val"].values.tolist(),
                  tdf[bErrorBounds[0][0]+1:bErrorBounds[0][1]]["SDC_loc"].values.tolist()))
bErrBefRec = list(zip(tdf[bErrorBounds[1][0]+1:bErrorBounds[1][1]]["SDC_val"].values.tolist(),
                    tdf[bErrorBounds[1][0]+1:bErrorBounds[1][1]]["SDC_loc"].values.tolist()))
bErrAftRec = list(zip(tdf[bErrorBounds[2][0]+1:bErrorBounds[2][1]]["SDC_val"].values.tolist(),
                     tdf[bErrorBounds[2][0]+1:bErrorBounds[2][1]]["SDC_loc"].values.tolist()))

#list(zip(tdf[bErrorBounds[2][0]+1:bErrorBounds[2][1]]['SDC_val'].values.tolist(),tdf[bErrorBounds[2][0]+1:bErrorBounds[2][1]]['SDC_loc'].values.tolist()))



#[i for i in bErrAftRec if i not in bErrBefDelay if i not in bErrBefRec if np.nan not in i]

In [107]:
runs_with_nvram_err = []
for run in nvsram2_2_half.runs.values():
    if run.aErrNVRAM:
        runs_with_nvram_err.append(run.num)
    elif run.bErrNVRAM:
        runs_with_nvram_err.append(run.num)
    elif run.cErrNVRAM:
        runs_with_nvram_err.append(run.num)
    elif run.dErrNVRAM:
        runs_with_nvram_err.append(run.num)

In [119]:
sram3_1_single_runs_ranked = []
sram3_1_half_runs_ranked   = []
sram3_2_half_ranked        = []
nvsram2_1_single_ranked    = []
nvsram2_1_half_ranked      = []
nvsram2_2_half_ranked      = []

for run in sram3_1_single.runs.values():
    if run.errors:
        sram3_1_single_runs_ranked.append((run.num, run.errors))

for run in sram3_1_half.runs.values():
    if run.errors:
        sram3_1_half_runs_ranked.append((run.num, run.errors))

for run in sram3_2_half.runs.values():
    if run.errors:
        sram3_2_half_ranked.append((run.num, run.errors))

for run in nvsram2_1_single.runs.values():
    if run.errors:
        nvsram2_1_single_ranked.append((run.num, run.errors))

for run in nvsram2_1_half.runs.values():
    if run.errors:
        nvsram2_1_half_ranked.append((run.num, run.errors))

for run in nvsram2_2_half.runs.values():
    if run.errors:
        nvsram2_2_half_ranked.append((run.num, run.errors))

sram3_1_single_runs_ranked.sort(reverse=True, key=lambda i: i[1])
sram3_1_half_runs_ranked.sort(reverse=True, key=lambda i: i[1])
sram3_2_half_ranked.sort(reverse=True, key=lambda i: i[1])
nvsram2_1_single_ranked.sort(reverse=True, key=lambda i: i[1])
nvsram2_1_half_ranked.sort(reverse=True, key=lambda i: i[1])
nvsram2_2_half_ranked.sort(reverse=True, key=lambda i: i[1])

In [122]:
# How many domiating 2nd differences are there in each run?

def diffFinder(variant):
    patterns = []
    
    #TODO: Check if most things with a pattern are flagged up by the below:
    for run in variant.values():    
        diffA = [int(run.aErrNVRAM[i+1][1], 16) - int(run.aErrNVRAM[i][1], 16) 
                        for i in range(0, len(run.aErrNVRAM)-1)]
        diffB = [int(run.bErrNVRAM[i+1][1], 16) - int(run.bErrNVRAM[i][1], 16)
                        for i in range(0, len(run.bErrNVRAM)-1)]
        diffC = [int(run.cErrNVRAM[i+1][1], 16) - int(run.cErrNVRAM[i][1], 16)
                        for i in range(0, len(run.cErrNVRAM)-1)]
        diffD = [int(run.dErrNVRAM[i+1][1], 16) - int(run.dErrNVRAM[i][1], 16)
                        for i in range(0, len(run.dErrNVRAM)-1)]

        diffA2 = [abs(diffA[i+1] - diffA[i]) for i in range(0, len(diffA)-1)]
        diffB2 = [abs(diffB[i+1] - diffB[i]) for i in range(0, len(diffB)-1)]
        diffC2 = [abs(diffC[i+1] - diffC[i]) for i in range(0, len(diffC)-1)]
        diffD2 = [abs(diffD[i+1] - diffD[i]) for i in range(0, len(diffD)-1)]

        diffADict1 = {k: v for k, v in sorted(Counter(diffA).items(), reverse=True, key=lambda i: i[1])}
        diffBDict1 = {k: v for k, v in sorted(Counter(diffB).items(), reverse=True, key=lambda i: i[1])}
        diffCDict1 = {k: v for k, v in sorted(Counter(diffC).items(), reverse=True, key=lambda i: i[1])}
        diffDDict1 = {k: v for k, v in sorted(Counter(diffD).items(), reverse=True, key=lambda i: i[1])}

        diffADict2 = {k: v for k, v in sorted(Counter(diffA2).items(), reverse=True, key=lambda i: i[1])}
        diffBDict2 = {k: v for k, v in sorted(Counter(diffB2).items(), reverse=True, key=lambda i: i[1])}
        diffCDict2 = {k: v for k, v in sorted(Counter(diffC2).items(), reverse=True, key=lambda i: i[1])}
        diffDDict2 = {k: v for k, v in sorted(Counter(diffD2).items(), reverse=True, key=lambda i: i[1])}

        if diffADict2:
            tot_err = len(run.aErrNVRAM)
            if list(diffADict2.items())[:1][0][1] >= 5:
                #if 128 not in diffADict2.keys():
                if (list(diffADict2.items())[:1][0][1] / tot_err) > 0.1:
                    patterns.append(run.num)
                #print(diffADict2)

        elif diffBDict2:
            tot_err = len(run.bErrNVRAM)
            if list(diffBDict2.items())[:1][0][1] >= 5:
                #if 128 not in diffBDict2.keys():
                if (list(diffBDict2.items())[:1][0][1] / tot_err) > 0.1:
                    patterns.append(run.num)
        
        elif diffCDict2:
            tot_err = len(run.cErrNVRAM)
            if list(diffCDict2.items())[:1][0][1] >= 5:
                #if 128 not in diffCDict2.keys():
                if (list(diffCDict2.items())[:1][0][1] / tot_err) > 0.1:
                    patterns.append(run.num)

        elif diffDDict2:
            tot_err = len(run.dErrNVRAM)
            if list(diffDDict2.items())[:1][0][1] >= 5:
                #if 128 not in diffDDict2.keys():
                if (list(diffDDict2.items())[:1][0][1] / tot_err) > 0.1:
                    patterns.append(run.num)
        else:
            #TODO: Check if there is a pattern in last digits of locations
            continue

    return patterns

    #if diffBDict2:
    #    if list(diffBDict2.items())[:1][0][1] >= 5:
    #        print(f"{run.num}: {diffBDict2}")
        #print(diffBDict2)

    #if diffCDict2:
    #    if list(diffCDict2.items())[:1][0][1] >= 5:
    #        print(f"{run.num}: {diffCDict2}") 
        #print(diffCDict2)
        #if list(diffCDict1.items())[:4][0][0] == 29:
            #print(list(diffCDict2.items())[:4])

    #if diffDDict2:
    #    if list(diffDDict2.items())[:1][0][1] >= 5:
    #        print(f"{run.num}: {diffDDict2}")
    #    print(diffDDict2)

diffFinder(sram3_1_half.runs)
#TODO: Throw away runs with patterns, i.e. throw away runs with comms errors.

[18,
 27,
 373,
 374,
 455,
 463,
 533,
 798,
 799,
 1014,
 1060,
 1061,
 1266,
 1418,
 1422,
 1423,
 1485,
 1486,
 1498,
 1499,
 1512,
 1547,
 1590,
 1593,
 1919,
 1920]

In [125]:
patterns = []

#for run in sram3_1_half.values():
for run in [sram3_1_half.runs[818]]:
    diffA = [int(run.aErrNVRAM[i+1][1], 16) - int(run.aErrNVRAM[i][1], 16) 
                for i in range(0, len(run.aErrNVRAM)-1)]
    diffB = [int(run.bErrNVRAM[i+1][1], 16) - int(run.bErrNVRAM[i][1], 16)
                for i in range(0, len(run.bErrNVRAM)-1)]
    diffC = [int(run.cErrNVRAM[i+1][1], 16) - int(run.cErrNVRAM[i][1], 16)
                for i in range(0, len(run.cErrNVRAM)-1)]
    diffD = [int(run.dErrNVRAM[i+1][1], 16) - int(run.dErrNVRAM[i][1], 16)
                for i in range(0, len(run.dErrNVRAM)-1)]

    diffA2 = [abs(diffA[i+1] - diffA[i]) for i in range(0, len(diffA)-1)]
    diffB2 = [abs(diffB[i+1] - diffB[i]) for i in range(0, len(diffB)-1)]
    diffC2 = [abs(diffC[i+1] - diffC[i]) for i in range(0, len(diffC)-1)]
    diffD2 = [abs(diffD[i+1] - diffD[i]) for i in range(0, len(diffD)-1)]

    diffADict1 = {k: v for k, v in sorted(Counter(diffA).items(), reverse=True, key=lambda i: i[1])}
    diffBDict1 = {k: v for k, v in sorted(Counter(diffB).items(), reverse=True, key=lambda i: i[1])}
    diffCDict1 = {k: v for k, v in sorted(Counter(diffC).items(), reverse=True, key=lambda i: i[1])}
    diffDDict1 = {k: v for k, v in sorted(Counter(diffD).items(), reverse=True, key=lambda i: i[1])}

    diffADict2 = {k: v for k, v in sorted(Counter(diffA2).items(), reverse=True, key=lambda i: i[1])}
    diffBDict2 = {k: v for k, v in sorted(Counter(diffB2).items(), reverse=True, key=lambda i: i[1])}
    diffCDict2 = {k: v for k, v in sorted(Counter(diffC2).items(), reverse=True, key=lambda i: i[1])}
    diffDDict2 = {k: v for k, v in sorted(Counter(diffD2).items(), reverse=True, key=lambda i: i[1])}

    tot_err = len(run.aErrNVRAM)



    # if diffADict2:
    #     if list(diffADict2.items())[:1][0][1] >= 5:
    #         #if 128 not in diffADict2.keys():
    #         if (list(diffADict2.items())[:1][0][1] / tot_err) > 0.1:
    #             patterns.append(run.num)
    #         #print(diffADict2)

    # elif diffBDict2:
    #     if list(diffBDict2.items())[:1][0][1] >= 5:
    #         #if 128 not in diffBDict2.keys():
    #         if (list(diffBDict2.items())[:1][0][1] / tot_err) > 0.1:
    #             patterns.append(run.num)
    
    # elif diffCDict2:
    #     if list(diffCDict2.items())[:1][0][1] >= 5:
    #         #if 128 not in diffCDict2.keys():
    #         if (list(diffCDict2.items())[:1][0][1] / tot_err) > 0.1:
    #             patterns.append(run.num)

    # elif diffDDict2:
    #     if list(diffDDict2.items())[:1][0][1] >= 5:
    #         #if 128 not in diffDDict2.keys():
    #         if (list(diffDDict2.items())[:1][0][1] / tot_err) > 0.1:
    #             patterns.append(run.num)
    # else:
    #     continue

In [ ]:
poo_arr = np.array([5, 2, 1])
poo_diff = np.abs(np.diff(poo_arr))
#np.absolute(np.diff(u8_arr))
Counter(poo_diff)

Need to deal with runs where experiment restarts during it

Finding runs with nvRAM errors:

In [ ]:
Counter([list(i) for i in zip(*sram3_1_half.runs[818].bErrorsNVRAM)][0])

In [ ]:
runErrorsRanked

Finding row errors:

In [ ]:
# test_run = sram3_1_half.runs[1593]
# differences  = []
# differences2 = []
# for i in range(0, len(test_run.cErrorsNVRAM)-1):
#     errLocNum = int(test_run.cErrorsNVRAM[i][1], 16)
#     nextErrLocNum = int(test_run.cErrorsNVRAM[i+1][1], 16)
#     #print(f"{nextErrLocNum}({test_run.errorListA[i+1][1]}) - {errLocNum}({test_run.errorListA[i][1]}) = {nextErrLocNum-errLocNum}")
#     firstDiff = nextErrLocNum - errLocNum
#     differences.append(firstDiff)
#     if firstDiff < 0:
#         print(f"{hex(nextErrLocNum)} - {hex(errLocNum)} = {firstDiff}")
# for i in range(0, len(differences)-1):
#     secondDiff = differences[i+1] - differences[i]
#     differences2.append(secondDiff)
#     if secondDiff > 0:
#         print(f"{differences[i+1]} - {differences[i]} = {secondDiff}")
# #differences2

In [ ]:
[i for i in sram3_1_half.runs[1485].cErrorsNVRAM]

In [ ]:
#test_run = sram3_1_half.runs[818]
test_run = sram3_1_half.runs[704]
differencesA = []
differencesB = []
differencesC = []
differencesD = []

rowLocs  = []
rowLocs2 = []
# for i in range(0, len(test_run.dErrorsNVRAM)-1):
#     differences.append(int(test_run.dErrorsNVRAM[i+1][1], 16) - int(test_run.dErrorsNVRAM[i][1], 16))

lastMemLocDigitA = [i[1][-1] for i in test_run.aErrorsNVRAM]
lastMemLocDigitB = [i[1][-1] for i in test_run.bErrorsNVRAM]
lastMemLocDigitC = [i[1][-1] for i in test_run.cErrorsNVRAM]
lastMemLocDigitD = [i[1][-1] for i in test_run.dErrorsNVRAM]

lMLDADiffs = [int(lastMemLocDigitA[i+1], 16) - int(lastMemLocDigitA[i], 16) for i in range(0, len(lastMemLocDigitA)-1)]
lMLDBDiffs = [int(lastMemLocDigitB[i+1], 16) - int(lastMemLocDigitB[i], 16) for i in range(0, len(lastMemLocDigitB)-1)]
lMLDCDiffs = [int(lastMemLocDigitC[i+1], 16) - int(lastMemLocDigitC[i], 16) for i in range(0, len(lastMemLocDigitC)-1)]
lMLDDDiffs = [int(lastMemLocDigitD[i+1], 16) - int(lastMemLocDigitD[i], 16) for i in range(0, len(lastMemLocDigitD)-1)]

#differencesB2 = [abs(differencesB[i+1] - differencesB[i]) for i in range(0, len(differencesB)-1)]
#differencesB2Dict = {k: v for k, v in sorted(Counter(differencesB2).items(), reverse=True, key=lambda i: i[1])}

# for i in range(0, len(test_run.aErrorsNVRAM)-1):
#     if int(test_run.cErrorsNVRAM[i+1][1], 16) - int(test_run.cErrorsNVRAM[i][1], 16) == 29:
#         rowLocs.append(test_run.cErrorsNVRAM[i])
#         rowLocs.append(test_run.cErrorsNVRAM[i+1])
#     if int(test_run.cErrorsNVRAM[i+1][1], 16) - int(test_run.cErrorsNVRAM[i][1], 16) == 99:
#         rowLocs2.append(test_run.cErrorsNVRAM[i])
#         rowLocs2.append(test_run.cErrorsNVRAM[i+1])

#differences2    = [abs(differences[i+1] - differences[i]) for i in range(0, len(differences)-1)]

#differences2Dict = Counter(differences2)
# differencesDict  = {k: v for k, v in sorted(Counter(differences).items(), reverse=True, key=lambda i: i[1])}
#differences2Dict = {k: v for k, v in sorted(differences2Dict.items(), reverse=True, key=lambda i: i[1])}

In [ ]:
test_runs = [sram3_1_half.runs[1593]]
#test_runs = [sram3_1_half.runs[818]]
test_runs = [sram3_1_half.runs[25]]

for run in test_runs:
    # lastMemLocDigitA = [i[1][-1] for i in run.aErrorsNVRAM]
    # lastMemLocDigitB = [i[1][-1] for i in run.bErrorsNVRAM]
    # lastMemLocDigitC = [i[1][-1] for i in run.cErrorsNVRAM]
    # lastMemLocDigitD = [i[1][-1] for i in run.dErrorsNVRAM]

    # lMLDADiffs = [int(lastMemLocDigitA[i+1], 16) - int(lastMemLocDigitA[i], 16) for i in range(0, len(lastMemLocDigitA)-1)]
    # lMLDBDiffs = [int(lastMemLocDigitB[i+1], 16) - int(lastMemLocDigitB[i], 16) for i in range(0, len(lastMemLocDigitB)-1)]
    # lMLDCDiffs = [int(lastMemLocDigitC[i+1], 16) - int(lastMemLocDigitC[i], 16) for i in range(0, len(lastMemLocDigitC)-1)]
    # lMLDDDiffs = [int(lastMemLocDigitD[i+1], 16) - int(lastMemLocDigitD[i], 16) for i in range(0, len(lastMemLocDigitD)-1)]

    # if lMLDADiffs:
    #     if '000000000' in ''.join(str(i) for i in lMLDADiffs):
    #         run.rowErr = 1
    # if lMLDBDiffs:
    #     if '000000000' in ''.join(str(i) for i in lMLDBDiffs):
    #         run.rowErr = 1
    # if lMLDCDiffs:
    #     if '000000000' in ''.join(str(i) for i in lMLDCDiffs):
    #         run.rowErr = 1
    # if lMLDDDiffs:
    #     if '000000000' in ''.join(str(i) for i in lMLDDDiffs):
    #         run.rowErr = 1

    # errorsA = {k: v for k, v in sorted(Counter([i[0] for i in run.aErrorsNVRAM]).items(), reverse=True, key=lambda i: i[1])}
    # errorsB = {k: v for k, v in sorted(Counter([i[0] for i in run.bErrorsNVRAM]).items(), reverse=True, key=lambda i: i[1])}
    # errorsC = {k: v for k, v in sorted(Counter([i[0] for i in run.cErrorsNVRAM]).items(), reverse=True, key=lambda i: i[1])}
    # errorsD = {k: v for k, v in sorted(Counter([i[0] for i in run.dErrorsNVRAM]).items(), reverse=True, key=lambda i: i[1])}

    diffA = [int(run.aErrorsNVRAM[i+1][1], 16) - int(run.aErrorsNVRAM[i][1], 16) 
                    for i in range(0, len(run.aErrorsNVRAM)-1)]
    diffB = [int(run.bErrorsNVRAM[i+1][1], 16) - int(run.bErrorsNVRAM[i][1], 16)
                    for i in range(0, len(run.bErrorsNVRAM)-1)]
    diffC = [int(run.cErrorsNVRAM[i+1][1], 16) - int(run.cErrorsNVRAM[i][1], 16)
                    for i in range(0, len(run.cErrorsNVRAM)-1)]
    diffD = [int(run.dErrorsNVRAM[i+1][1], 16) - int(run.dErrorsNVRAM[i][1], 16)
                    for i in range(0, len(run.dErrorsNVRAM)-1)]

    diffA2 = [abs(diffA[i+1] - diffA[i]) for i in range(0, len(diffA)-1)]
    diffB2 = [abs(diffB[i+1] - diffB[i]) for i in range(0, len(diffB)-1)]
    diffC2 = [abs(diffC[i+1] - diffC[i]) for i in range(0, len(diffC)-1)]
    diffD2 = [abs(diffD[i+1] - diffD[i]) for i in range(0, len(diffD)-1)]

    diffADict1 = {k: v for k, v in sorted(Counter(diffA).items(), reverse=True, key=lambda i: i[1])}
    diffBDict1 = {k: v for k, v in sorted(Counter(diffB).items(), reverse=True, key=lambda i: i[1])}
    diffCDict1 = {k: v for k, v in sorted(Counter(diffC).items(), reverse=True, key=lambda i: i[1])}
    diffDDict1 = {k: v for k, v in sorted(Counter(diffD).items(), reverse=True, key=lambda i: i[1])}

    diffADict2 = {k: v for k, v in sorted(Counter(diffA2).items(), reverse=True, key=lambda i: i[1])}
    diffBDict2 = {k: v for k, v in sorted(Counter(diffB2).items(), reverse=True, key=lambda i: i[1])}
    diffCDict2 = {k: v for k, v in sorted(Counter(diffC2).items(), reverse=True, key=lambda i: i[1])}
    diffDDict2 = {k: v for k, v in sorted(Counter(diffD2).items(), reverse=True, key=lambda i: i[1])}

    #differencesAMode = [i for i in differencesADict.keys() if ]
    #differencesBMode = [differencesBDict.keys()[i] for i in range(0, len(differencesBDict.keys())) if differencesBDict.keys()[i] + differencesBDict.keys()[i] > l]

    try:
        if list(diffADict2.values())[0]/(len(diffA)-1) > 0.1:
            print(f"\n{list(diffADict2.values())[0]/(len(diffA)-1)}")
            print("pooA")
    except IndexError:
        print("A wrong")

    try:
        if list(diffBDict2.values())[0]/(len(diffB)-1) > 0.1:
            print(f"\n{list(diffBDict2.values())[0]/(len(diffB)-1)}")
            print("pooB") 
    except IndexError:
        print("B wrong")

    try:   
        if list(diffCDict2.values())[0]/(len(diffC)-1) > 0.1:
            print(f"\n{list(diffCDict2.values())[0]/(len(diffC)-1)}") 
            print("pooC")
    except IndexError:
        print("C wrong")

    try:
        if list(diffDDict2.values())[0]/(len(diffD)-1) > 0.1:
            print(f"\n{list(diffDDict2.values())[0]/(len(diffD)-1)}") 
            print("pooD")
    except IndexError:
        print("D wrong")
    


In [ ]:
sram3_1_half.runs[818].bErrorsNVRAM

In [ ]:
runErrorsRankedNew = []
for run in sram3_1_half.runs.values():
    if run.errors & run.rowErr:
        runErrorsRankedNew.append((run.num, run.errors))

runErrorsRankedNew.sort(reverse=True, key=lambda i: i[1])

In [ ]:
testrun = sram3_1_half.runs[1593]
tlastMemLocDigitA = [i[1][-1] for i in testrun.aErrorsNVRAM]
tlastMemLocDigitB = [i[1][-1] for i in testrun.bErrorsNVRAM]
tlastMemLocDigitC = [i[1][-1] for i in testrun.cErrorsNVRAM]
tlastMemLocDigitD = [i[1][-1] for i in testrun.dErrorsNVRAM]
tlMLDADiffs = [int(tlastMemLocDigitA[i+1], 16) - int(tlastMemLocDigitA[i], 16) for i in range(0, len(tlastMemLocDigitA)-1)]
tlMLDBDiffs = [int(tlastMemLocDigitB[i+1], 16) - int(tlastMemLocDigitB[i], 16) for i in range(0, len(tlastMemLocDigitB)-1)]
tlMLDCDiffs = [int(tlastMemLocDigitC[i+1], 16) - int(tlastMemLocDigitC[i], 16) for i in range(0, len(tlastMemLocDigitC)-1)]
tlMLDDDiffs = [int(tlastMemLocDigitD[i+1], 16) - int(tlastMemLocDigitD[i], 16) for i in range(0, len(tlastMemLocDigitD)-1)]

# if tlMLDADiffs:
#     if '000000000' in ''.join(str(i) for i in tlMLDADiffs):
#         print("poo")
# if tlMLDBDiffs:
#     if '000000000' in ''.join(str(i) for i in tlMLDADiffs):
#         print("poo")
# if tlMLDCDiffs:
#     if '000000000' in ''.join(str(i) for i in tlMLDADiffs):
#         print("poo")
# if tlMLDDDiffs:
#     if '000000000' in ''.join(str(i) for i in tlMLDADiffs):
#         print("poo")

In [ ]:
for i in range(0, len(test_run.bErrorsNVRAM)-1):
    print(test_run.bErrorsNVRAM[i])
    print(f"    {differencesB[i]}")

Plot the number of flipped bits per neutron event across the current data.

In [ ]:
goldbyte = 170
hamDistList = []
ignores = [i[0] for i in runErrorsRanked[0:35]]
for run in sram3_1_half.runs.values():
    if run.num not in ignores:
        if run.errors:
            for error in run.aErrorsNVRAM:
                hamDist = bin(int(error[0], 16) ^ goldbyte).count('1')
                hamDistList.append(hamDist)

hamDistDict = Counter(hamDistList)
plt.bar(hamDistDict.keys(), hamDistDict.values())
plt.title("Number of flipped bits per neutron event")
plt.xlabel("Flipped bits")
plt.ylabel("Single event effects")

In [ ]:
sram3_1_half.runs[704].dErrorsNVRAM